In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import pdb
from tqdm import tqdm

import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import pairwise_distances
import jellyfish

import joblib
from joblib import Parallel, delayed

np.random.seed(97)

In [2]:
final_design_seqs_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/final_design/"
ensemble_predictions_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/ensemble_predictions/"

In [3]:
# NOTE: ensemble 0 is the original ensemble 3 and ensemble 1 is the original ensemble 4
ensemble_1_filtered_sequences = pd.read_parquet(os.path.join(ensemble_predictions_dir, 
                                                             "filtered_ensemble_4_predictions.parquet"))
ensemble_1_dataset_seqs = ensemble_1_filtered_sequences[ensemble_1_filtered_sequences["design_method"] == "Dataset"].reset_index(drop=True)

In [4]:
methods = ["coms", "dens"]
num_seqs = [None, None]
all_seqs = []
for i, method in enumerate(methods):
    print(method, num_seqs[i])
    seqs = pickle.load(open(os.path.join(final_design_seqs_dir, f"{method}_final_design.pkl"), "rb"))
    seqs_df = []
    kmer_features = {}
    for cell in ["jurkat", "k562", "thp1"]:
        kmer_features[cell] = seqs[cell]["kmer_features"]
        seqs[cell].pop("kmer_features")
        df = pd.DataFrame(seqs[cell])
        df["designed_for"] = cell
        
        if num_seqs[i] is not None:
            df = df.tail(num_seqs[i])
        
        seqs_df.append(df)
    
    seqs_df = pd.concat(seqs_df).reset_index(drop=True)
    all_seqs.extend(seqs_df["sequences"])

coms None
dens None


In [ ]:
raw_seqs = list(ensemble_1_dataset_seqs["sequence"]) + all_seqs
print(len(raw_seqs))
pairwise_edit_distances = Parallel(n_jobs=-1)(delayed(jellyfish.damerau_levenshtein_distance)(raw_seqs[i], raw_seqs[j]) 
                                              for i in tqdm(range(len(raw_seqs))) 
                                              for j in range(i+1, len(raw_seqs), 1))
np.save(os.path.join(final_design_seqs_dir, f"pairwise_damerau_levenshtein_distances"), pairwise_edit_distances)
np.save(os.path.join(final_design_seqs_dir, f"raw_seqs"), raw_seqs)

46310


  1%|          | 291/46310 [11:02<29:20:23,  2.30s/it]